In [18]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [15]:
cap = cv2.VideoCapture(0)

face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

skip=0
face_data=[]

dataset_path='./data/'

file_name = input("Enter the name of person:")
while True:
    
    ret,frame = cap.read()
    if ret==False:
        continue
    fray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    #list of faces is "faces"
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    faces=sorted(faces,key=lambda f:f[2]*f[3])
    
    for face in faces[-1:]:
        x,y,w,h=face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

        #Cropping the required part
        offset=10
        face_section= frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        
        skip+=1
        
        if(skip%10==0):
            face_data.append(face_section)
            print(len(face_data))
    
    cv2.imshow("Frame",frame)
    cv2.imshow("Face section" , face_section)

    if(skip%10==0):
        pass 
    
    key_pressed = cv2.waitKey(1) & 0xFF
    if(key_pressed == ord('q')):
        break
        
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0]),-1)
print(face_data.shape)
        
np.save(dataset_path+file_name+'.npy',face_data)
print("Data Saved : " ,dataset_path+file_name+'.npy')
cap.release()
cv2.destroyAllWindows()


Enter the name of person:deepanshu
1
2
3
4
5
6
7
8
9
(9, 30000)
Data Saved :  ./data/deepanshu.npy


In [30]:
def distance(x1,x2):
        return np.sqrt(sum((x1-x2)**2))
def knn(train, test, k=5):
	dist = []
	
	for i in range(train.shape[0]):
		# Get the vector and label
		ix = train[i, :-1]
		iy = train[i, -1]
		# Compute the distance from test point
		d = distance(test, ix)
		dist.append([d, iy])
	# Sort based on distance and get top k
	dk = sorted(dist, key=lambda x: x[0])[:k]
	# Retrieve only the labels
	labels = np.array(dk)[:, -1]
	
	# Get frequencies of each label
	output = np.unique(labels, return_counts=True)
	# Find max frequency and corresponding label
	index = np.argmax(output[1])
	return output[0][index]

In [33]:
cap = cv2.VideoCapture(0)

face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

skip=0
face_data=[]
labels=[]

class_id=0
names={}
dataset_path='./data/'



for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        
        #mapping
        names[class_id]=fx[:-4]
        
        print("loaded"+fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
    
        target = class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)

face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

print(face_labels.shape)
print(face_dataset.shape)

trainset=np.concatenate((face_dataset,face_labels),axis=1)
# 1 column for label will be added
print(trainset.shape)


loadeddeepanshu.npy
(9, 1)
(9, 30000)
(9, 30001)


In [34]:
while True:
    ret,frame =cap.read()
    if ret==False:
        continue
    
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    
    for face in faces:
        x,y,w,h=face
        
        offset=10
        face_section = frame[y-offset:y+offset,x-offset:x+offset]
        face_section = cv2.resize(face_section,(100,100)) 
        
        out = knn(trainset,face_section.flatten())
        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
    cv2.imshow("Faces",frame)
    
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()